In [1]:
import PlottingFunction as pf
import DataManipulation as dm
import matplotlib.pyplot as plt
from pathlib import Path
import pandas as pd
from sklearn.cluster import KMeans
import winsound

%matplotlib qt5

In [2]:
df = pd.read_csv(Path.cwd() / 'data' / 'vast' / "vast_rally_600k.csv")
df = dm.keep_data_in_frame(df, compressed=True, only_half=True)

In [3]:
receiver_position = 'receiver_start'
receiver_position_title = "Start"
n_clusters = 6
overlap = False
means = {}
impact_regions, receiver_regions = pf.createPlayerRegions(type = 'rally')

for impact_region in impact_regions:
    plt.figure()
    i = 0
    for receiver_region in receiver_regions:
        df_filter = df
        df_filter = dm.filter_data_by_buckets(df_filter, 'impact', impact_regions[impact_region], compressed=True)
        df_filter = dm.filter_data_by_buckets(df_filter, receiver_position, receiver_regions[receiver_region], compressed=True)

        if overlap == True:
            cluster = KMeans(n_clusters=n_clusters, random_state=0).fit(df_filter[['x02', 'y02', 'px0', 'py0', 'ox0', 'oy0']])
        else:
            cluster = KMeans(n_clusters=n_clusters, random_state=0).fit(df_filter[['x02', 'y02']])

        centroids = cluster.cluster_centers_
        df_filter["Cluster"] = cluster.labels_

        means_tmp = df_filter.groupby('Cluster')['vast'].mean()
        means[impact_region, receiver_region] = means_tmp

        plt.subplot(1, 3, 1+i)
        pf.plot1D(df_filter, centroids, means_tmp, alpha_ball = 0.50, receiver_position=receiver_position, just_ball=True, compressed=True, only_half=True, 
                  title =  "Clustered Rallies: Impact Region (" + impact_region + ") | Receiver " + receiver_position_title + " Region (" + receiver_region +")")
        pf.markupImpactPlayer(impact_region)
        pf.markupCourt(receiver_position,receiver_regions[receiver_region], compressed=True)
        pf.caption_cluster_plots(df_filter, centroids, n_clusters)
        i=i+1

    plt.subplots_adjust(top=0.7,bottom=0.2,left=0.04,right=0.97,hspace=0.15,wspace=0.17)

winsound.Beep(2000, 800)